In [ ]:
import sqlite3
import pandas as pd
import json
from sentence_transformers import SentenceTransformer, util
import torch

In [ ]:
# Подключение к SQLite базе данных
conn = sqlite3.connect('/repo/converted_database.db')

# Загрузка данных из таблицы OKPD_2
query = "SELECT OKPD2, OKPD2_NAME FROM OKPD_2"
okpd2_data = pd.read_sql_query(query, conn)
conn.close()

# Загрузка дерева категорий из JSON файла
with open('/repo/category_tree.json', 'r') as f:
    market_data = json.load(f)

In [ ]:
def extract_categories(data, parent_names=None):
    """
    Рекурсивная функция для извлечения категорий и их идентификаторов из 
    иерархической структуры JSON.

    Args:
        data (dict): Данные JSON с информацией о категориях.
        parent_names (list, optional): Список названий родительских категорий.

    Returns:
        list: Список словарей, содержащих информацию о категориях.
    """
    records = []
    if parent_names is None:
        parent_names = []
    
    # Проверяем наличие дочерних категорий
    if 'children' in data:
        for child in data['children']:
            new_parent_names = parent_names + [data['name']]
            records.extend(extract_categories(child, new_parent_names))
    else:
        # Сохраняем информацию о категории
        category_record = {'type_name': data['name'], 'type_id': data['id']}
        for i, parent_name in enumerate(parent_names):
            category_record[f'level_{i+1}_category'] = parent_name
        records.append(category_record)
    
    return records

In [ ]:
# Извлечение записей категорий из JSON
market_records = extract_categories(market_data['result'])
market_df = pd.DataFrame(amrket_records)

# Загрузка предобученной модели для преобразования предложений в векторы
model = SentenceTransformer('deepvk/USER-bge-m3').to('cuda')

# Получение названий из данных OKPD2 и Market
okpd2_names = okpd2_data['OKPD2_NAME'].tolist()
market_names = market_df['type_name'].tolist()

# Преобразование названий в векторы с использованием модели
okpd2_embeddings = model.encode(okpd2_names, convert_to_tensor=True, device='cuda')
market_embeddings = model.encode(market_names, convert_to_tensor=True, device='cuda')

# Вычисление косинусного сходства между векторами
cosine_scores = util.pytorch_cos_sim(okpd2_embeddings, market_embeddings)

# Нахождение лучших совпадений на основе косинусного сходства
best_matches_cosine = []
similarity_scores = []

In [ ]:
for i in range(len(okpd2_names)):
    best_index = cosine_scores[i].argmax()
    best_matches_cosine.append(market_names[best_index])
    similarity_scores.append(cosine_scores[i][best_index].item())

# Добавление результатов в DataFrame
okpd2_data['best_match'] = best_matches_cosine
okpd2_data['cosine_similarity'] = similarity_scores

# Объединение данных OKPD2 с соответствующими категориями Yandex Market
columns_to_merge = ['type_name'] + [col for col in market_df.columns if 'level_' in col]
result_df = okpd2_data.merge(market_df[columns_to_merge], 
                              left_on='best_match', 
                              right_on='type_name', 
                              how='left')

# Сохранение результатов в новый CSV файл
output_file = '/repo/okpd2_best_matches.csv'
columns_to_save = ['OKPD2', 'OKPD2_NAME'] + [col for col in result_df.columns if 'level_' in col] + ['best_match', 'cosine_similarity']
result_df[columns_to_save].to_csv(output_file, index=False, encoding='utf-8-sig')

print(f"Результаты сохранены в {output_file}")